In [1]:
import os
import flickrapi

api_key = os.environ.get('FLICKR_KEY')
api_secret = os.environ.get('FLICKR_SECRET')

flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [2]:
flickr.get_request_token(oauth_callback='oob')

do_request: Status code 401 received, content:
    oauth_problem=signature_invalid
    debug_sbs=POST
    https://www.flickr.com/services/oauth/request_token
    oauth_callback=oob&oauth_consumer_key=fb5239d737a0070a19d065876c29ba42&oauth_nonce=177812919211233530131484433242&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1484433242&oauth_token=72157677649630750-b7a3875a5d04fe13&oauth_version=1.0


FlickrError: do_request: Status code 401 received

In [4]:
print('Step 1: authenticate')

# Only do this if we don't have a valid token already
if not flickr.token_valid(perms='read'):

    # Get a request token
    flickr.get_request_token(oauth_callback='oob')

    # Open a browser at the authentication URL. Do this however
    # you want, as long as the user visits that URL.
    authorize_url = flickr.auth_url(perms='read')
    print(authorize_url)

Step 1: authenticate
https://www.flickr.com/services/oauth/authorize?oauth_token=72157679022966726-47687ee3a3a9c596&perms=read


In [ ]:
verifier = str(input('Verifier code: '))

# Trade the request token for an access token
flickr.get_access_token(verifier)

print('Step 2: use Flickr')
resp = flickr.photos.getInfo(photo_id='7658567128')

Step 1: authenticate
Verifier code: 667-766-226


NameError: name 'resp' is not defined